**Modelo predictivo: recomendación académica de los estudiantes de la Universidad Metropolitana**

Universidad Metropolitana

Proyecto de Investigación de Ingeniería de Sistemas

PREPROCESAMIENTO DE DATOS


In [0]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

## Funciones auxiliares para el preprocesamiento de la data

**Clasificación de las notas**

Se verifica la nota del estudiante y se clasifica de la siguiente manera:

1.   Mal: si obtuvo una nota entre 10 y 12.
2.   Bien: si obtuvo una nota de 13 en adelante, APRO, SOB (Servicio Com.), SOBR (TTG), o NOT (Servicio Com. y TTG)
3. Reprobo: si reprobó u obtuvo DIFE
4. Retiro: si retiro la materia

In [0]:
def asignar_clas(x):
    if(x == '#'):
        return 'R'
    elif(x == 'NREQ'):
        return 'NREQ'
    elif (x == 'SOB' or x == 'SOBR' or x == 'NOT' or x == 'APRO' or x == 'CONT'):
      return 'Bien'
    elif(x == 'DIFE' or x == 'REPR'):
        return 'Reprobo'
    elif(int(x) < 10):
        return 'Reprobo'
    elif(int(x) < 13):
        return 'Mal'
    elif(int(x) <= 20): 
        return 'Bien'  

In [0]:
# Función para separar la materia de Servicio Comunitario en dos: inicial y continua
def separar_sc(x):
    if(x == 'CONT'):
        return 'BPTDI01-1'
    else:
        return 'BPTDI01-2'

In [0]:
# Se filtra el dataset para trabajar solo con las materias de ingeniería de sistemas
def filtrar_materias(notas):    
    ing_sistemas = {'FBPCE03','FBPCE04','FBPLI01','FBPLI02','FBPMM01','FBPMM02',
                    'FBPIN01','FBPIN02','FBPIN03',
                    'FBTMM00','FBTHU01','FBTCE03','FBTIE02','FBTIN04',
                    'FBTMM01','FBTHU02','FBTCE04','FBTIE03','FBTIN05',
                    'BPTMI01','BPTPI07','BPTQI21','FBTSP03',
                    'BPTMI02','BPTMI30','BPTQI22','BPTFI01',
                    'BPTMI03','BPTSP05','BPTFI02',
                    'BPTMI04','BPTMI31','BPTSP06','BPTEN12','BPTFI03',
                    'BPTMI11','FPTSP04','BPTSP03','BPTFI04',
                    'BPTMI05','FPTSP17','FPTSP01','BPTSP04','FPTEN23',
                    'FBTIE01','FPTSP19','FPTSP07','FPTSP02','BPTMI20',
                    'FPTPI09','FPTSP21','FPTSP20','FPTMI21','FPTSP22',
                    'FPTSP11','FPTSP18','FPTSP23',
                    'FPTSP14','FPTSP15','FPTCS16','FPTSP25','FPTIS01',
                    'FGE0000','FGS0000','BPTDI01-1','BPTDI01-2'}
    
    temp_notas = notas
    mask = temp_notas['asignatura'].isin(ing_sistemas)
    temp_notas = temp_notas[mask]
    return temp_notas


## Preprocesamiento de los datos

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Se unifican los históricos de los estudiantes de cada trimestre en un solo conjunto

In [0]:
notas = pd.DataFrame()
for filename in os.listdir('drive/Shared drives/Tesis/Proyecto de investigación/Modelo 1/Datos/CSV'):
    tmp = pd.read_csv('drive/Shared drives/Tesis/Proyecto de investigación/Modelo 1/Datos/CSV/' + filename, encoding='latin-1')
    tmp['trimestre'] = filename.split('.')[0]
    tmp = tmp[['asignatura','id', 'nombre_asignatura', 'nota', 'trimestre']]
    notas = notas.append(tmp,sort=True)

notas = notas.sort_values(by=['trimestre', 'id'])

Aplicación de distintas técnicas de preprocesamiento al conjunto de datos unificado: completar datos vacíos, unificar materias electivas y de selección respectivamente, clasificación de las notas y eliminación de filas no relevantes.

In [0]:
electivas = {'FGE': 'FGE', 'FGS': 'FGS'}
notas['Electiva'] = notas['asignatura'].transform(lambda x: x[:3])
notas['Electiva'] = notas['Electiva'].map(electivas)
notas['Electiva'] = notas['Electiva'].fillna('NO')

#Reemplazar los campos vacíos en la columna notas por #
notas['nota'] = notas['nota'].fillna('#')

#Separar la materia servicio comunitario en 2: BPTDI01-1 BPTDI01-2
notas['asignatura'][notas['asignatura'] == 'BPTDI01'] = notas['nota'][notas['asignatura'] == 'BPTDI01'].transform(lambda x: separar_sc(x))

#Unificar las materias electivas y de selección en un solo código de asignatura respectivamente
notas['asignatura'][notas['Electiva'] == 'FGE'] = 'FGE0000'
notas['asignatura'][notas['Electiva'] == 'FGS'] = 'FGS0000'

#Añadir una columna indicando la clasificación de la nota obtenida
notas['clasificacion_nota'] = notas['nota'].transform(lambda x: asignar_clas(x))
notas = notas.drop(notas[notas['clasificacion_nota'] == 'NREQ'].index)

#Eliminar las materias que no corresponden a la carrera de Ingeniería de Sistemas
notas = filtrar_materias(notas)

#Establecer el nombre de la columna a 
notas['one_hot'] = notas['asignatura'] + '_' + notas['clasificacion_nota']

notas = notas.drop(['Electiva'], axis=1)

#Exportar en csv los datos preprocesados
# export_to_csv = notas.to_csv('notas_new_classes.csv')
# !cp notas_new_classes.csv drive/Shared\ drives/Tesis/Proyecto\ de\ investigación/Modelo\ 5/Datos


In [0]:
# Mostrar ejemplo de la data preprocesada para un estudiante
pd.set_option('display.max_rows', 100)
notas[notas['id']==10090939]

### Implementación del one hot encoding de notas de los estudiantes por materias

Obtención de las columnas del one hot

In [0]:
one_hot = []
asignaturas = notas.asignatura.unique()
asignaturas = np.sort(asignaturas)

for asignatura in asignaturas:

    for clas in notas.clasificacion_nota.unique():
      
        if(asignatura!= None and clas != None):
            one_hot.append(asignatura+ '_' + clas)
one_hot = np.sort(one_hot)

In [0]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

Se aplica la técnica de one hot encoding para representar los valores de las notas de cada estudiante en las distintas materias cursadas, estableciendo 1 en cada columna correspondiente a la materia cursada y la clasificación de nota obtenida por el mismo (ejemplo: BPTMI01_Bien), y 0 en las restantes.

In [0]:
df_train = pd.DataFrame()

total_trim = notas.trimestre.unique().size
i_trim = 0

out_trim = display(progress(0, total_trim), display_id=True)
out_estudiante = display(progress(0, 100), display_id=True)

for trimestre in notas.trimestre.unique():
    out_trim.update(progress(i_trim, total_trim))
    
    total_estudiante = notas['id'].unique().size
    i_estudiante = 0
    
    for estudiante in notas['id'][notas['trimestre']==trimestre].unique():
        out_estudiante.update(progress(i_estudiante, total_estudiante))
        tmp = notas[notas['trimestre'] == trimestre]
        row = {}
        row['trimestre'] = trimestre
        row['estudiante'] = estudiante
        for i in one_hot:
            row[i] = 0
        for index, tupla in tmp[tmp['id'] == estudiante].iterrows():
            materia = tupla['one_hot']
            row[materia]=1
        #print(row)
        tmp2 = pd.DataFrame(row,index=[0])
        df_train = df_train.append(tmp2)
        i_estudiante += 1
    i_trim += 1

# Exportar el one hot encoding de las notas de los estudiantes en csv
# export_to_csv = df_train.to_csv('one_hot_new_classes.csv')
# !cp one_hot_new_classes.csv drive/Shared\ drives/Tesis/Proyecto\ de\ investigación/Modelo\ 5/Datos